In [1]:
import io
import glob
import pandas as pd
import numpy
import collections
from csv import reader

In [11]:
#pour lister les absolute path des fichiers à traiter
corpusde = glob.glob("/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/*.txt")
#print(corpusfr)
len(corpusde) #on vérifie combien il y a de fichiers

275

In [12]:
#pour lister les textes en virant les scories qui troublent les noms de colonnes
corpusde_propre =[]
k=0
while k<len(corpusde):
    corpusde_propre.append(corpusde[k].replace('/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/*.txt',''))
    k+=1
#print(corpusde_propre)

In [4]:
nomsdic = pd.read_csv("/home/angelique/Documents/these/reperage_EN/noms7.csv", sep='\t')

nomsdic.head

<bound method NDFrame.head of             nom
0        Arbois
1     Ardaillon
2           Ary
3      Aymonier
4       Babelon
...         ...
4197       Spon
4198     Carter
4199       Hase
4200  Marcellus
4201    Biardot

[4202 rows x 1 columns]>

In [5]:
listenoms = [] #création d'une liste de noms à insérer plus tard dans les résultats. On prend le temps de supprimer des \n fâcheux
with open("/home/angelique/Documents/these/reperage_EN/noms7.csv", 'r') as f:
    for line in f:
        v = str(line)
        v = v.replace('\n','')
        listenoms.append(v)
listenoms.pop(0) #on supprime l'index du tableau puisqu'on remettra encore un nouvel index dans le tableau des résultats

'nom'

In [6]:
 #création d'un hashset avec tous les noms
noms = set((nomsdic['nom'].values)) 

In [7]:
def reperagenoms(listeNoms, corpusde):
    results = pd.DataFrame()
    k=0
    ponctuation = [',','.',':',';','!','?','"','»',')','(','„','»','«','”']
    listeNoms = [[] for i in range(len(corpusde))] #on fait une liste de liste. chaque liste contiendra le nom cherché puis le nombre d'occurence de chaque nom
    results = pd.DataFrame() #on crée un df qui contiendra les résultats
    results.insert(loc = 0, column='nom de personnages', value = listenoms) #on met dans le tableau des résultats la première colonne qui est les noms de personnage
    while k < len(corpusde):
        infile = open(corpusde[k],encoding = 'utf-8')
        #print(corpus[k])
        text = infile.read()
        infile.close()
        for punct in ponctuation:
            text = text.replace(punct, ' '+punct)
        #text = text.title() #on fait en sorte que tous les mots commencent par une majuscule (comme dans la liste des noms qu'on cherche) en fait non sinon on a du bruit comme des cons
        texte_liste = text.split() #On transforme le texte en liste de mots séparés par des espaces
        texte_noms = [token for token in texte_liste if token in noms] #on stripe les textes des mots qui ne font pas partie de ma liste de noms (but alléger le code)
        compte = collections.Counter(texte_noms) #dictionnaire avec en clé un nom et en valeur le support associé (la fréquence quoi, on parle l'académique ici)
        #print(compte.most_common(5)) #on lui fait imprimer les compteurs des 5 max de la ligne
        liste_compte = [compte[mot] for mot in nomsdic['nom'].values] #liste compréhension qui entre une colonne avec les comptages
        results.insert(loc = k+1,column = corpusde_propre[k], value = liste_compte)
        k+=1
    return results

In [13]:
listeNoms = []

results = reperagenoms(listeNoms, corpusde) 

/home/angelique/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
results['freq']=results.sum(axis=1) #on crée une colonne fréquence avec la somme
results.sort_values('freq',ascending=False) #on classe par ordre décroissant
#on se prend un performance warning mais on s'en fiche ça marche

<ipython-input-14-17ac9e29c904>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results['freq']=results.sum(axis=1) #on crée une colonne fréquence avec la somme


,nom de personnages,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1872_2.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1914_2.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/BCH1886.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/BCH1896.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/letronne1885_corr.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/seure1906_corr.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1884_1.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/ridder1901_corr.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1877_1.txt,...,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1906_2.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1894_2.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/BCH1877.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/daremberg4_1_1873.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1898_1.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/gaz_arch1886.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/rev_arch1903_2.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/collignon1898_corr.txt,/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/wagnon1885_corr.txt,freq
147,Reinach,0,36,20,44,0,0,15,9,0,...,47,28,0,6,22,24,88,2,0,3846
664,Perrot,6,45,15,4,0,0,12,4,2,...,10,23,4,1,8,3,11,0,33,2513
321,Conze,9,6,5,9,7,0,8,9,11,...,13,13,5,6,19,18,13,3,3,2428
517,Lenormant,6,6,3,7,14,0,12,3,1,...,2,1,3,0,10,24,7,2,2,2163
92,Homolle,0,5,18,42,0,3,1,0,3,...,5,18,6,0,12,11,4,5,0,2067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,"""Correa""""""",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
410,Glavinic',0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3455,Halbherrs,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3456,Stüdniczka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
results.to_csv(r'/home/angelique/Documents/these/reperage_EN/results_fr_221116.csv') #on exporte